## 尝试对目的层曲线进行截断

**这个方法只对直井有效**

部分曲线目的层顶部或底部缺少分层解释，根据砂厚的统计信息，选最大值进行截断

In [1]:
import os
from pathlib import Path

import lasio
import numpy as np
import pandas as pd


In [2]:
# 定义路径
las_folder = "../data/vertical_well_common_las"
horizon_file = "../data/well_horizon_processed.xlsx"
thickness_stats_file = "../data/sand_group_thickness_complete.xlsx"
output_folder = "../data/vertical_well_truncated_las"

# 创建输出文件夹
output_path = Path(output_folder)
output_path.mkdir(parents=True, exist_ok=True)

# 定义砂组层序（从上到下）
sand_groups = [
    "H1-1",
    "H2-1",
    "H3Ding(23D)",
    "H4ding(34D)",
    "H5ding",
    "H6ding(53D)",
    "H7ding(63D)",
    "H8ding(72D)",
    "P0(H83D)",
]

# 读取层位数据
print("=" * 60)
print("读取层位数据...")
df_horizon = pd.read_excel(horizon_file)
print(f"层位数据: {len(df_horizon)} 条记录")

# 读取厚度统计数据（Statistics表）
print("\n读取厚度统计数据...")
df_stats = pd.read_excel(thickness_stats_file, sheet_name="Statistics")
print(f"厚度统计数据:")
print(df_stats)

# 提取最大厚度值，创建字典 {层段名: 最大厚度}
max_thickness = {}
for _, row in df_stats.iterrows():
    sand_group = row['Sand Group']
    max_val = row['Max (m)']
    max_thickness[sand_group] = max_val

print(f"\n各层段最大厚度:")
for k, v in max_thickness.items():
    print(f"  {k}: {v:.2f} m")

# 获取所有LAS文件
input_path = Path(las_folder)
las_files = list(input_path.glob("*.las")) + list(input_path.glob("*.LAS"))
las_files = list(set(las_files))  # 去重
las_files.sort()

if not las_files:
    print(f"\n在 {las_folder} 中未找到LAS文件")
else:
    print(f"\n找到 {len(las_files)} 个LAS文件")
    print("=" * 60)

    # 处理每个LAS文件
    success_count = 0
    failed_files = []

    for las_file in las_files:
        well_name = las_file.stem  # 不含扩展名的文件名

        print(f"\n处理井: {well_name}")

        try:
            # 读取LAS文件
            las = lasio.read(las_file, mnemonic_case="upper")

            # 获取该井的层位数据
            well_horizons = df_horizon[df_horizon['Well'] == well_name].copy()

            if len(well_horizons) == 0:
                print(f"  ⚠️ 在层位数据中未找到井 {well_name}，跳过")
                failed_files.append((well_name, "无层位数据"))
                continue

            # 筛选出该井存在的砂组层位
            well_sand_groups = well_horizons[well_horizons['Surface'].isin(sand_groups)].copy()

            if len(well_sand_groups) == 0:
                print(f"  ⚠️ 井 {well_name} 没有任何目标砂组层位，跳过")
                failed_files.append((well_name, "无目标层位"))
                continue

            # 创建层位MD字典
            surface_md = {}
            for _, row in well_sand_groups.iterrows():
                surface_md[row['Surface']] = row['MD']

            existing_surfaces = list(surface_md.keys())
            print(f"  该井存在的层位: {existing_surfaces}")

            # 确定该井最浅和最深的层位
            existing_indices = [sand_groups.index(s) for s in existing_surfaces]
            first_idx = min(existing_indices)
            last_idx = max(existing_indices)

            first_surface = sand_groups[first_idx]
            last_surface = sand_groups[last_idx]

            print(f"  最浅层位: {first_surface} (索引 {first_idx})")
            print(f"  最深层位: {last_surface} (索引 {last_idx})")

            # 计算顶界深度
            if first_idx == 0:
                # 如果最浅层位就是H1-1，直接使用其MD
                top_md = surface_md[first_surface]
                print(f"  顶界MD: {top_md:.2f} m (使用 {first_surface} 实际MD)")
            else:
                # 需要估算，从最浅层位向上减去缺失层段的最大厚度
                top_md = surface_md[first_surface]

                # 缺失的层段是从 0 到 first_idx-1
                # 对应的厚度名称是 H1 Thickness 到 H{first_idx} Thickness
                missing_thickness = 0
                for i in range(first_idx):
                    thickness_name = f"H{i+1} Thickness"
                    thickness_val = max_thickness.get(thickness_name, 0)
                    missing_thickness += thickness_val
                    print(f"    - 缺失层段 {thickness_name}: {thickness_val:.2f} m")

                top_md = top_md - missing_thickness - 50
                print(f"  估算顶界MD: {top_md:.2f} m (减去 {missing_thickness:.2f} m)")

            # 计算底界深度
            if last_idx == len(sand_groups) - 1:
                # 如果最深层位就是P0(H83D)，直接使用其MD
                bottom_md = surface_md[last_surface]
                print(f"  底界MD: {bottom_md:.2f} m (使用 {last_surface} 实际MD)")
            else:
                # 需要估算，从最深层位向下加上缺失层段的最大厚度
                bottom_md = surface_md[last_surface]

                # 缺失的层段是从 last_idx 到 len(sand_groups)-2
                # 对应的厚度名称是 H{last_idx+1} Thickness 到 H{len(sand_groups)-1} Thickness
                missing_thickness = 0
                for i in range(last_idx, len(sand_groups) - 1):
                    thickness_name = f"H{i+1} Thickness"
                    thickness_val = max_thickness.get(thickness_name, 0)
                    missing_thickness += thickness_val
                    print(f"    - 缺失层段 {thickness_name}: {thickness_val:.2f} m")

                bottom_md = bottom_md + missing_thickness + 50
                print(f"  估算底界MD: {bottom_md:.2f} m (加上 {missing_thickness:.2f} m)")

            # 获取深度曲线（通常是DEPT或DEPTH）
            depth_curve = None
            for mnemonic in ['DEPT', 'DEPTH', 'MD']:
                if mnemonic in [c.mnemonic for c in las.curves]:
                    depth_curve = mnemonic
                    break

            if depth_curve is None:
                print(f"  ✗ 未找到深度曲线，跳过")
                failed_files.append((well_name, "无深度曲线"))
                continue

            print(f"  使用深度曲线: {depth_curve}")

            # 获取深度数据
            depth_data = las[depth_curve]

            # 找到截断范围的索引
            mask = (depth_data >= top_md) & (depth_data <= bottom_md)

            if not mask.any():
                print(f"  ✗ 截断范围内没有数据点，跳过")
                print(f"     深度范围: {depth_data.min():.2f} - {depth_data.max():.2f} m")
                print(f"     截断范围: {top_md:.2f} - {bottom_md:.2f} m")
                failed_files.append((well_name, "截断范围无数据"))
                continue

            truncated_count = mask.sum()
            original_count = len(depth_data)
            print(f"  截断: {original_count} -> {truncated_count} 个数据点")

            # 创建新的LAS对象
            new_las = lasio.LASFile()
            new_las.version = las.version
            new_las.well = las.well
            new_las.params = las.params
            new_las.other = las.other

            # 添加截断后的曲线数据
            for curve in las.curves:
                truncated_data = curve.data[mask]
                new_las.append_curve(
                    mnemonic=curve.mnemonic,
                    data=truncated_data,
                    unit=curve.unit,
                    descr=curve.descr,
                    value=curve.value
                )

            # 保存新文件
            output_file = output_path / las_file.name
            new_las.write(str(output_file), version=2.0)

            print(f"  ✓ 成功截断并保存到 {output_file.name}")
            success_count += 1

        except Exception as e:
            print(f"  ✗ 处理失败: {e}")
            import traceback
            traceback.print_exc()
            failed_files.append((well_name, str(e)))

    # 输出统计信息
    print("\n" + "=" * 60)
    print(f"处理完成！")
    print(f"成功: {success_count}/{len(las_files)}")

    if failed_files:
        print(f"\n失败的文件:")
        for filename, error in failed_files:
            print(f"  - {filename}: {error}")


读取层位数据...
层位数据: 726 条记录

读取厚度统计数据...
厚度统计数据:
     Sand Group  Average Thickness (m)  Std Dev (m)  Min (m)  Max (m)
0  H1 Thickness                130.130    15.816450     95.9    160.2
1  H2 Thickness                 87.570    10.100552     78.1    120.7
2  H3 Thickness                 99.080     7.688173     84.0    112.7
3  H4 Thickness                 99.580    12.254177     76.7    118.3
4  H5 Thickness                 95.230    23.681996     71.0    160.8
5  H6 Thickness                117.465    10.141124     92.7    144.0
6  H7 Thickness                 73.055    20.841267     52.7    155.4
7  H8 Thickness                 59.250    12.847343     35.7     80.3

各层段最大厚度:
  H1 Thickness: 160.20 m
  H2 Thickness: 120.70 m
  H3 Thickness: 112.70 m
  H4 Thickness: 118.30 m
  H5 Thickness: 160.80 m
  H6 Thickness: 144.00 m
  H7 Thickness: 155.40 m
  H8 Thickness: 80.30 m

找到 7 个LAS文件

处理井: PH1
  该井存在的层位: ['H3Ding(23D)', 'H4ding(34D)', 'H1-1', 'H2-1', 'H5ding', 'H6ding(53D)', 'H7ding(63